In [1]:
import tensorflow as tf
import cv2
from timeit import default_timer as timer
from moviepy.editor import VideoFileClip

D:\UCU\Course_3\AI\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import glob,os
import matplotlib.image as mpimg
from scipy.misc import imresize
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from keras.models import model_from_json
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
# %load visuaizations.py
import cv2
import numpy as np

def draw_lane_status(frame, lane_info, threshold_offset = 0.6):

    font = cv2.FONT_HERSHEY_SIMPLEX
    info_road = "Lane Status"
    info_lane = "Direction: {0}".format(lane_info['curve_direction'])
    info_cur = "Curvature {:6.1f} m".format(lane_info['curvature'])
    info_offset = "Off center: {0} {1:3.1f}m".format(lane_info['dev_dir'], lane_info['offset'])

    l_uper = (10,10)

    cv2.line(frame,(l_uper[0] + 265,0),(l_uper[0] + 265,155),(255,0,0),5)

    cv2.putText(frame, info_road, (50,32+5), font, 0.8, (255,255,0), 2,cv2.LINE_AA)
    cv2.putText(frame, info_lane, (16,60+10), font, 0.6, (255,255,0), 1,cv2.LINE_AA)
    cv2.putText(frame, info_cur, (16,80+25), font, 0.6, (255,255,0), 1,cv2.LINE_AA)

    if lane_info['offset'] >= threshold_offset:
        cv2.putText(frame, info_offset, (16,100+40), font, 0.6, (255,0,0), 1,cv2.LINE_AA)
    else:
        cv2.putText(frame, info_offset, (16,100+40), font, 0.6, (255,255,0), 1,cv2.LINE_AA)

def draw_speed(img_cp, fps, w):

    fps_info = "{0:4.1f} fps".format(fps)
    cv2.putText(img_cp, 'Speed', (w - 120,37), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2, cv2.LINE_AA)
    cv2.putText(img_cp, fps_info, (w - 130,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 1, cv2.LINE_AA)
    cv2.line(img_cp,(w-160,0),(w-160,155),(255,0,0),5)


def draw_thumbnails(img_cp, img, window_list, thumb_w=100, thumb_h=80, off_x=30, off_y=30):

    cv2.putText(img_cp, 'Detected viehicles', (400,37), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2, cv2.LINE_AA)
    for i, bbox in enumerate(window_list):
        thumbnail = img[bbox[0][1]:bbox[1][1], bbox[0][0]:bbox[1][0]]
        vehicle_thumb = cv2.resize(thumbnail, dsize=(thumb_w, thumb_h))
        start_x = 300 + (i+1) * off_x + i * thumb_w
        img_cp[off_y + 30:off_y + thumb_h + 30, start_x:start_x + thumb_w, :] = vehicle_thumb


def draw_background_highlight(image, draw_img, w):

    mask = cv2.rectangle(np.copy(image), (0, 0), (w, 155), (0, 0, 0), thickness=cv2.FILLED)
    draw_img = cv2.addWeighted(src1=mask, alpha=0.3, src2=draw_img, beta=0.8, gamma=0)

    return draw_img

In [4]:
# %load "python prog/yolo_pipeline.py"
# import numpy as np
import tensorflow as tf
# import cv2
from timeit import default_timer as timer
# import time
# import matplotlib.pyplot as plt

class yolo_tf:
    w_img = 1280
    h_img = 720

    weights_file = 'YOLO_small.ckpt'
    alpha = 0.1
    threshold = 0.3
    iou_threshold = 0.5

    result_list = None
    classes =  ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair",
                "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant",
                "sheep", "sofa", "train","tvmonitor"]

    def __init__(self):
        self.build_networks()

    def build_networks(self):
        print("Building YOLO_small graph...")
        self.x = tf.placeholder('float32',[None,448,448,3])
        # self.x = tf.placeholder('float32',[None,252, 1280, 3])
        self.conv_1 = self.conv_layer(1,self.x,64,7,2)
        self.pool_2 = self.pooling_layer(2,self.conv_1,2,2)
        self.conv_3 = self.conv_layer(3,self.pool_2,192,3,1)
        self.pool_4 = self.pooling_layer(4,self.conv_3,2,2)
        self.conv_5 = self.conv_layer(5,self.pool_4,128,1,1)
        self.conv_6 = self.conv_layer(6,self.conv_5,256,3,1)
        self.conv_7 = self.conv_layer(7,self.conv_6,256,1,1)
        self.conv_8 = self.conv_layer(8,self.conv_7,512,3,1)
        self.pool_9 = self.pooling_layer(9,self.conv_8,2,2)
        self.conv_10 = self.conv_layer(10,self.pool_9,256,1,1)
        self.conv_11 = self.conv_layer(11,self.conv_10,512,3,1)
        self.conv_12 = self.conv_layer(12,self.conv_11,256,1,1)
        self.conv_13 = self.conv_layer(13,self.conv_12,512,3,1)
        self.conv_14 = self.conv_layer(14,self.conv_13,256,1,1)
        self.conv_15 = self.conv_layer(15,self.conv_14,512,3,1)
        self.conv_16 = self.conv_layer(16,self.conv_15,256,1,1)
        self.conv_17 = self.conv_layer(17,self.conv_16,512,3,1)
        self.conv_18 = self.conv_layer(18,self.conv_17,512,1,1)
        self.conv_19 = self.conv_layer(19,self.conv_18,1024,3,1)
        self.pool_20 = self.pooling_layer(20,self.conv_19,2,2)
        self.conv_21 = self.conv_layer(21,self.pool_20,512,1,1)
        self.conv_22 = self.conv_layer(22,self.conv_21,1024,3,1)
        self.conv_23 = self.conv_layer(23,self.conv_22,512,1,1)
        self.conv_24 = self.conv_layer(24,self.conv_23,1024,3,1)
        self.conv_25 = self.conv_layer(25,self.conv_24,1024,3,1)
        self.conv_26 = self.conv_layer(26,self.conv_25,1024,3,2)
        self.conv_27 = self.conv_layer(27,self.conv_26,1024,3,1)
        self.conv_28 = self.conv_layer(28,self.conv_27,1024,3,1)
        self.fc_29 = self.fc_layer(29,self.conv_28,512,flat=True,linear=False)
        self.fc_30 = self.fc_layer(30,self.fc_29,4096,flat=False,linear=False)
        #skip dropout_31
        self.fc_32 = self.fc_layer(32, self.fc_30, 1470, flat=False, linear=True)
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        self.saver.restore(self.sess, self.weights_file)
        print("Loading complete!")

    def conv_layer(self,idx,inputs,filters,size,stride):
        channels = inputs.get_shape()[3]
        weight = tf.Variable(tf.truncated_normal([size,size,int(channels),filters], stddev=0.1))
        biases = tf.Variable(tf.constant(0.1, shape=[filters]))

        pad_size = size//2
        pad_mat = np.array([[0,0],[pad_size,pad_size],[pad_size,pad_size],[0,0]])
        inputs_pad = tf.pad(inputs,pad_mat)

        conv = tf.nn.conv2d(inputs_pad, weight, strides=[1, stride, stride, 1], padding='VALID',name=str(idx)+'_conv')
        conv_biased = tf.add(conv,biases,name=str(idx)+'_conv_biased')
        print('Layer  %d : Type = Conv, Size = %d * %d, Stride = %d, Filters = %d, Input channels = %d' % (idx,size,size,stride,filters,int(channels)))
        return tf.maximum(self.alpha*conv_biased,conv_biased,name=str(idx)+'_leaky_relu')

    def pooling_layer(self,idx,inputs,size,stride):
        print ('Layer  %d : Type = Pool, Size = %d * %d, Stride = %d' % (idx,size,size,stride))
        return tf.nn.max_pool(inputs, ksize=[1, size, size, 1],strides=[1, stride, stride, 1], padding='SAME',name=str(idx)+'_pool')

    def fc_layer(self,idx,inputs,hiddens,flat = False,linear = False):
        input_shape = inputs.get_shape().as_list()
        if flat:
            dim = input_shape[1]*input_shape[2]*input_shape[3]
            inputs_transposed = tf.transpose(inputs,(0,3,1,2))
            inputs_processed = tf.reshape(inputs_transposed, [-1,dim])
        else:
            dim = input_shape[1]
            inputs_processed = inputs
        weight = tf.Variable(tf.truncated_normal([dim,hiddens], stddev=0.1))
        biases = tf.Variable(tf.constant(0.1, shape=[hiddens]))
        print ('Layer  %d : Type = Full, Hidden = %d, Input dimension = %d, Flat = %d, Activation = %d' % (idx,hiddens,int(dim),int(flat),1-int(linear))	)
        if linear : return tf.add(tf.matmul(inputs_processed,weight),biases,name=str(idx)+'_fc')
        ip = tf.add(tf.matmul(inputs_processed,weight),biases)
        return tf.maximum(self.alpha*ip,ip,name=str(idx)+'_fc')

def detect_from_cvmat(yolo,img):
    yolo.h_img,yolo.w_img,_ = img.shape
    img_resized = cv2.resize(img, (448, 448))
    img_resized_np = np.asarray( img_resized )
    inputs = np.zeros((1,448,448,3),dtype='float32')
    inputs[0] = (img_resized_np/255.0)*2.0-1.0
    in_dict = {yolo.x: inputs}
    net_output = yolo.sess.run(yolo.fc_32,feed_dict=in_dict)

    result = interpret_output(yolo, net_output[0])
    yolo.result_list = result


def detect_from_file(yolo,filename):
    detect_from_cvmat(yolo, filename)


def interpret_output(yolo,output):
    probs = np.zeros((7,7,2,20))
    class_probs = np.reshape(output[0:980],(7,7,20))
    scales = np.reshape(output[980:1078],(7,7,2))
    boxes = np.reshape(output[1078:],(7,7,2,4))
    offset = np.transpose(np.reshape(np.array([np.arange(7)]*14),(2,7,7)),(1,2,0))

    boxes[:,:,:,0] += offset
    boxes[:,:,:,1] += np.transpose(offset,(1,0,2))
    boxes[:,:,:,0:2] = boxes[:,:,:,0:2] / 7.0
    boxes[:,:,:,2] = np.multiply(boxes[:,:,:,2],boxes[:,:,:,2])
    boxes[:,:,:,3] = np.multiply(boxes[:,:,:,3],boxes[:,:,:,3])

    boxes[:,:,:,0] *= yolo.w_img
    boxes[:,:,:,1] *= yolo.h_img
    boxes[:,:,:,2] *= yolo.w_img
    boxes[:,:,:,3] *= yolo.h_img

    for i in range(2):
        for j in range(20):
            probs[:,:,i,j] = np.multiply(class_probs[:,:,j],scales[:,:,i])

    filter_mat_probs = np.array(probs>=yolo.threshold,dtype='bool')
    filter_mat_boxes = np.nonzero(filter_mat_probs)
    boxes_filtered = boxes[filter_mat_boxes[0],filter_mat_boxes[1],filter_mat_boxes[2]]
    probs_filtered = probs[filter_mat_probs]
    classes_num_filtered = np.argmax(filter_mat_probs,axis=3)[filter_mat_boxes[0],filter_mat_boxes[1],filter_mat_boxes[2]]

    argsort = np.array(np.argsort(probs_filtered))[::-1]
    boxes_filtered = boxes_filtered[argsort]
    probs_filtered = probs_filtered[argsort]
    classes_num_filtered = classes_num_filtered[argsort]

    for i in range(len(boxes_filtered)):
        if probs_filtered[i] == 0 : continue
        for j in range(i+1,len(boxes_filtered)):
            if iou(boxes_filtered[i],boxes_filtered[j]) > yolo.iou_threshold :
                probs_filtered[j] = 0.0

    filter_iou = np.array(probs_filtered>0.0,dtype='bool')
    boxes_filtered = boxes_filtered[filter_iou]
    probs_filtered = probs_filtered[filter_iou]
    classes_num_filtered = classes_num_filtered[filter_iou]

    result = []
    for i in range(len(boxes_filtered)):
        result.append([yolo.classes[classes_num_filtered[i]],boxes_filtered[i][0],boxes_filtered[i][1],boxes_filtered[i][2],boxes_filtered[i][3],probs_filtered[i]])

    return result


def draw_results(img,  yolo, fps):
    img_cp = img.copy()
    results = yolo.result_list

    # draw the highlighted background
   # img_cp = draw_background_highlight(img_cp, image_lane, yolo.w_img)

    window_list = []
    for i in range(len(results)):
        x = int(results[i][1])
        y = int(results[i][2])
        w = int(results[i][3])//2
        h = int(results[i][4])//2
        cv2.rectangle(img_cp,(x-w,y-h),(x+w,y+h),(0,0,255),4)
        cv2.rectangle(img_cp,(x-w,y-h-20),(x+w,y-h),(125,125,125),-1)
        # cv2.putText(img_cp,results[i][0] + ' : %.2f' % results[i][5],(x-w+5,y-h-7),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0),1)
        cv2.putText(img_cp,results[i][0],(x-w+5,y-h-7),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0),1)
        if results[i][0] == "car" or results[i][0] == "bus":
            window_list.append(((x-w,y-h),(x+w,y+h)))

    # draw vehicle thumbnails
    try:
        draw_thumbnails(img_cp, img, window_list)
    except:
        pass

    # draw speed
    # draw_speed(img_cp, fps, yolo.w_img)

    # draw lane status
    #draw_lane_status(img_cp,lane_info)

    return img_cp

def iou(box1,box2):
    tb = min(box1[0]+0.5*box1[2],box2[0]+0.5*box2[2])-max(box1[0]-0.5*box1[2],box2[0]-0.5*box2[2])
    lr = min(box1[1]+0.5*box1[3],box2[1]+0.5*box2[3])-max(box1[1]-0.5*box1[3],box2[1]-0.5*box2[3])
    if tb < 0 or lr < 0 : intersection = 0
    else : intersection =  tb*lr
    return intersection / (box1[2]*box1[3] + box2[2]*box2[3] - intersection)



yolo = yolo_tf()

def vehicle_detection_yolo(image):
    # set the timer
    start = timer()
    detect_from_file(yolo, image)

    # compute frame per second
    fps = 1.0 / (timer() - start)
    # draw visualization on frame
    yolo_result = draw_results(image, yolo, fps)

    return yolo_result


Building YOLO_small graph...
Layer  1 : Type = Conv, Size = 7 * 7, Stride = 2, Filters = 64, Input channels = 3
Layer  2 : Type = Pool, Size = 2 * 2, Stride = 2
Layer  3 : Type = Conv, Size = 3 * 3, Stride = 1, Filters = 192, Input channels = 64
Layer  4 : Type = Pool, Size = 2 * 2, Stride = 2
Layer  5 : Type = Conv, Size = 1 * 1, Stride = 1, Filters = 128, Input channels = 192
Layer  6 : Type = Conv, Size = 3 * 3, Stride = 1, Filters = 256, Input channels = 128
Layer  7 : Type = Conv, Size = 1 * 1, Stride = 1, Filters = 256, Input channels = 256
Layer  8 : Type = Conv, Size = 3 * 3, Stride = 1, Filters = 512, Input channels = 256
Layer  9 : Type = Pool, Size = 2 * 2, Stride = 2
Layer  10 : Type = Conv, Size = 1 * 1, Stride = 1, Filters = 256, Input channels = 512
Layer  11 : Type = Conv, Size = 3 * 3, Stride = 1, Filters = 512, Input channels = 256
Layer  12 : Type = Conv, Size = 1 * 1, Stride = 1, Filters = 256, Input channels = 512
Layer  13 : Type = Conv, Size = 3 * 3, Stride = 1, 

In [5]:
def pipeline_yolo(img):


    #img_undist, img_lane_augmented, lane_info = lane_process(img)

    output = vehicle_detection_yolo(img)
    output_res = imresize(output, (720, 1280, 3))
    

    return output_res

#### Line detection

In [6]:
# Load pretrained model

# load the model json file 
json_file = open('model.json', 'r')
json_model = json_file.read()
json_file.close()

# load the model from the json
model = model_from_json(json_model)
# load the wieghts of the model
model.load_weights('model.h5')

In [7]:
# create a class Lane 

# Class to average lanes with
class Lanes():
    def __init__(self):
        self.recent_fit = []
        self.avg_fit = []

In [8]:
# resize the image and predict the lane to be drawn from the model in G color

def road_lines_image(image):
    
    img_arr = image
    actual_image = imresize(img_arr, (720, 1280, 3))

    # Get image ready for feeding into model
    img = image
    small_img_2 = imresize(img, (80, 160, 3))
    
    mask = np.zeros((small_img_2.shape[0], small_img_2.shape[1]), dtype="uint8")
    pts = np.array([[0, 80], [0, 40], [160, 40], [160, 80]], dtype=np.int32)
    cv2.fillConvexPoly(mask, pts, 255)
    
    masked = cv2.bitwise_and(small_img_2, small_img_2, mask=mask)
    small_img = masked[None, :, :, :]

    # Make prediction with neural network (un-normalize value by multiplying by 255)
    prediction = model.predict(small_img)[0] * 255

    lanes = Lanes()
    
    # Add lane prediction to list for averaging
    lanes.recent_fit.append(prediction)
    # Only using last five for average
    if len(lanes.recent_fit) > 5:
        lanes.recent_fit = lanes.recent_fit[1:]

    # Calculate average detection
    lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis = 0)

    # Generate fake R & B color dimensions, stack with G
    blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)
    lane_drawn = np.dstack((blanks, lanes.avg_fit, blanks))

    # Re-size to match the original image
    lane_image = imresize(lane_drawn, (720, 1280, 3))

    # Merge the lane drawing onto the original image
#     result = cv2.addWeighted(actual_image, 1, lane_image, 1, 0)

    return lane_image

#### Processing

In [9]:
def proccess_frame(frame):
    

    img_lines = road_lines_image(frame)
    img = pipeline_yolo(frame)
    
    result = cv2.addWeighted(img_lines, 1, img, 1, 0)
    return result
   
   
   

In [10]:
# name = "qwe"

# cv2.namedWindow(name)
# cap=cv2.VideoCapture('my_test.mp4')

# if cap.isOpened():
    
#     ret, frame = cap.read()
# else: 
#     ret = False
    
# while ret: 
#     ret, frame = cap.read()
    
   
           
#     result = proccess_frame(frame)
    
#     cv2.imshow(name, result)

    
#     if cv2.waitKey(1) == 27: 
#         break
# cv2.destroyWindow(name)

# cap.release()

In [11]:
counter=0
output = 'final_result_3.mp4'
clip1 = VideoFileClip("my_test.mp4")
#clip1 = VideoFileClip("project_video.mp4").subclip(22,25)
#clip1 = VideoFileClip("challenge_video.mp4")
#clip1 = VideoFileClip("challenge_video.mp4")

#clip1.save_frame("frame.jpeg")
#clip1 = clip1.fx(mve.vfx.rotate, lambda t: 90*t, expand=False)
out_clip = clip1.fl_image(proccess_frame) #NOTE: this function expects color images!!
%time out_clip.write_videofile(output, audio=False, verbose = True)
print(counter)

[MoviePy] >>>> Building video final_result_3.mp4
[MoviePy] Writing video final_result_3.mp4


100%|██████████████████████████████████████████████████████████████████████████████| 2833/2833 [33:11<00:00,  1.55it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: final_result_3.mp4 

Wall time: 33min 12s
0
